In [1]:
! pip install --upgrade pip
! python -c "import pykeen" || pip install git+https://github.com/pykeen/pykeen.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pykeen'
  Cloning https://github.com/pykeen/pykeen.git to /private/var/folders/8q/qqsmchbj7_d42k8qmj55vj180000gn/T/pip-req-build-6a59_w7v
  Running command git clone --filter=blob:none --quiet https://github.com/pykeen/pykeen.git /private/var/folders/8q/qqsmchbj7_d42k8qmj55vj180000gn/T/pip-req-build-6a59_w7v
  Resolved https://github.com/pykeen/pykeen.git to commit 42bfa99a17ee489d0d54dc6332d5cc71fc0ded08
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.

In [2]:
import torch
import pykeen
import pandas as pd
from pykeen import predict
from pykeen.datasets import Nations
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory

In [6]:
from rdflib import Graph

g = Graph()
g.parse("B2_research_publications_abox.ttl", format="ttl")
with open("B2_research_publications_abox.tsv", "w", encoding="utf-8") as f:
    f.write("subject\tpredicate\tobject\n")  # Header row
    for subj, pred, obj in g:
        f.write(f"{subj}\t{pred}\t{obj}\n")


In [7]:
file_path = 'B2_research_publications_abox.tsv'
tf = TriplesFactory.from_path(file_path)
training, testing = tf.split()
device = "cuda" if torch.cuda.is_available() else "cpu"

result = pipeline(
    training=training,
    testing=testing,
    model="TransE",
    model_kwargs=dict(
        embedding_dim=128,
    ),
    training_kwargs=dict(
        num_epochs=20
    ),
    optimizer_kwargs=dict(
        lr=0.01,
    ),
    negative_sampler_kwargs=dict(
        num_negs_per_pos=1,
    ),
    random_seed=2025,
    device = device
)

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [26341, 12074]
INFO:pykeen.pipeline.api:Using device: cpu
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


Training epochs on cpu:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0.00/189 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/189 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/189 [00:00<?, ?batch/s]

...

Training batches on cpu:   0%|          | 0.00/189 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/189 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0.00/189 [00:00<?, ?batch/s]

Evaluating on cpu:   0%|          | 0.00/12.1k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 73.92s seconds


In [12]:
evaluation = result.metric_results.to_dict()
evaluation['both']['realistic']

{'standard_deviation': 5116.8046875,
 'adjusted_inverse_harmonic_mean_rank': 0.0672301746477257,
 'median_rank': 998.0,
 'adjusted_arithmetic_mean_rank_index': 0.7811608034603846,
 'z_arithmetic_mean_rank': 209.67341979769293,
 'median_absolute_deviation': 1470.7414007575571,
 'z_geometric_mean_rank': 149.0793196003827,
 'adjusted_arithmetic_mean_rank': 0.21889415142360513,
 'inverse_geometric_mean_rank': 0.002298144856467843,
 'arithmetic_mean_rank': 3111.4892578125,
 'variance': 26181692.0,
 'harmonic_mean_rank': 14.795634783933446,
 'inverse_median_rank': 0.0010020040208473802,
 'adjusted_geometric_mean_rank_index': 0.9583703729095195,
 'inverse_harmonic_mean_rank': 0.06758750230073929,
 'count': 24148.0,
 'inverse_arithmetic_mean_rank': 0.0003213894960936159,
 'z_inverse_harmonic_mean_rank': 1370.3866646504766,
 'geometric_mean_rank': 435.1335754394531,
 'hits_at_1': 0.011802219645519297,
 'hits_at_3': 0.1013748550604605,
 'hits_at_5': 0.12551764121252276,
 'hits_at_10': 0.15612058

In [8]:
evaluation = result.metric_results.to_dict()
evaluation['both']['realistic']

{'standard_deviation': 5116.8046875,
 'adjusted_inverse_harmonic_mean_rank': 0.0672301746477257,
 'median_rank': 998.0,
 'adjusted_arithmetic_mean_rank_index': 0.7811608034603846,
 'z_arithmetic_mean_rank': 209.67341979769293,
 'median_absolute_deviation': 1470.7414007575571,
 'z_geometric_mean_rank': 149.0793196003827,
 'adjusted_arithmetic_mean_rank': 0.21889415142360513,
 'inverse_geometric_mean_rank': 0.002298144856467843,
 'arithmetic_mean_rank': 3111.4892578125,
 'variance': 26181692.0,
 'harmonic_mean_rank': 14.795634783933446,
 'inverse_median_rank': 0.0010020040208473802,
 'adjusted_geometric_mean_rank_index': 0.9583703729095195,
 'inverse_harmonic_mean_rank': 0.06758750230073929,
 'count': 24148.0,
 'inverse_arithmetic_mean_rank': 0.0003213894960936159,
 'z_inverse_harmonic_mean_rank': 1370.3866646504766,
 'geometric_mean_rank': 435.1335754394531,
 'hits_at_1': 0.011802219645519297,
 'hits_at_3': 0.1013748550604605,
 'hits_at_5': 0.12551764121252276,
 'hits_at_10': 0.15612058

In [9]:
result.training.relation_to_id

{'': 0,
 'http://example.org/research#cites': 1,
 'http://example.org/research#hasEdition': 2,
 'http://example.org/research#hasKeyword': 3,
 'http://example.org/research#hasReview': 4,
 'http://example.org/research#heldDuring': 5,
 'http://example.org/research#heldInCity': 6,
 'http://example.org/research#publishedIn': 7,
 'http://example.org/research#reviewer': 8,
 'http://example.org/research#reviewsPaper': 9,
 'http://example.org/research#title': 10,
 'http://example.org/research#writes': 11,
 'http://example.org/research#year': 12,
 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type': 13,
 'predicate': 14}

In [10]:
result.training.entity_to_id

{'': 0,
 ' POSTCARDIOTOMY SYNDROME , ANTICOAGULANTS, AND HÆMOPERICARDIUM': 1,
 '$B_s \\rightarrow D_s^*$ Form Factors for the full $q^2$ range from Lattice QCD': 2,
 "'. L. Nee": 3,
 "'. Sallyguttormsen": 4,
 "'市客厅'的感悟--上海人民广场评析": 5,
 '(Anti-)de Sitter Black Hole Entropy and Generalized Uncertainty Principle': 6,
 '-asymmetries based on an ': 7,
 '-flavor analysis': 8,
 '02社-27-口-20 中学校における武道必修化によって期待される教育効果 : 教員の立場から(02.体育社会学,一般研究発表抄録)': 9,
 '1,1′-Binaphthol annulated perylene diimides: Aggregation-induced emission enhancement and chirality inversion': 10,
 '1247 The Doppler Velocimetry in Twin Pregnancy Complicated by Intrauterine Growth Restriction is Predictive of Neonatal Outcome|[quest]|': 11,
 '124P Phase II study of neoadjuvant concurrent chemo-immuno-radiation therapy followed by surgery and adjuvant immunotherapy for resectable stage IIIA-B N2 non-small cell lung cancer: SQUAT trial (WJOG 12119L)': 12,
 '1542P First real-world outcome data of SCLC in Germany: Data from the Cl

In [13]:

model_name = "KGE_B2_research_publications_model1"
result.save_to_directory(model_name)


INFO:pykeen.triples.triples_factory:Stored TriplesFactory(num_entities=29509, num_relations=15, create_inverse_triples=False, num_triples=48294, path="/Users/ceciliaperez/Documents/UPC-MD/Semestre 4/SDM/P2_sdm/sdm_knowledge_graphs/B2_research_publications_abox.tsv") to file:///Users/ceciliaperez/Documents/UPC-MD/Semestre%204/SDM/P2_sdm/sdm_knowledge_graphs/KGE_B2_research_publications_model1/training_triples
INFO:pykeen.pipeline.api:Saved to directory: /Users/ceciliaperez/Documents/UPC-MD/Semestre 4/SDM/P2_sdm/sdm_knowledge_graphs/KGE_B2_research_publications_model1
